<h1> Updated Traffic from LTA News </h1>

The objective of this project is to get the updated news from LTA twitter website to pinpoint the location of accident.<br />
First, we will need to prepare the data from the twitter. I have use selenium to parse more tweets than given 1 page.

In [5]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

browser = webdriver.Chrome()
url = "https://twitter.com/ltatrafficnews"

browser.get(url)
time.sleep(2)
'''
# when internet connection is good use the first way to scroll down
SCROLL_PAUSE_TIME = 5


# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
'''
body = browser.find_element_by_tag_name('body')
for _ in range(1000):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    
tweets = browser.find_elements_by_class_name('tweet-text')
time_stamps = browser.find_elements_by_class_name('js-short-timestamp')

t_list = []
for tweet in tweets:
    t_list.append(tweet.text)

ts_list = []
for ts in time_stamps:
    ts_list.append(ts.get_attribute('data-time'))
    
browser.quit()

'''
import requests
from io import StringIO
from lxml import html
import HTMLParser
from datetime import datetime

webpage = requests.get("https://twitter.com/ltatrafficnews")
doc = html.fromstring(webpage.content)
tweet_container = doc.xpath('//p[@class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text"]/text()')
#print(tweet_container)
temp = doc.xpath('//span[contains(@class,"_timestamp js-short-timestamp js-relative-timestamp")]')
temp = [t.get('data-time') for t in temp]
temp = [int(t) for t in temp]
temp = [datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S') for ts in temp]
#print(temp)
'''

'\nimport requests\nfrom io import StringIO\nfrom lxml import html\nimport HTMLParser\nfrom datetime import datetime\n\nwebpage = requests.get("https://twitter.com/ltatrafficnews")\ndoc = html.fromstring(webpage.content)\ntweet_container = doc.xpath(\'//p[@class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text"]/text()\')\n#print(tweet_container)\ntemp = doc.xpath(\'//span[contains(@class,"_timestamp js-short-timestamp js-relative-timestamp")]\')\ntemp = [t.get(\'data-time\') for t in temp]\ntemp = [int(t) for t in temp]\ntemp = [datetime.fromtimestamp(ts).strftime(\'%Y-%m-%d %H:%M:%S\') for ts in temp]\n#print(temp)\n'

Next, we are create a dataframe from the data that we have fetch off the twitter

In [6]:
from datetime import datetime
import pandas as pd
import numpy as np

def formfetch(tweets,timestamps):
    if len(tweets) == 0 or len(timestamps) == 0:
        print("There's no data successfully fetched from chrome, check the driver and element of webpage.")
        df = None
    else:
        tweets = [t.encode('utf-8') for t in tweets]
        times = [int(t) for t in timestamps]
        times = [datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S') for ts in times]
        df = pd.DataFrame(np.column_stack([times,tweets]),columns=['DateTime','Event'])
        #df.to_csv("test_csv.csv",encoding='utf-8',index=False)
        print("Successfully fetch data from twitter.")
    return df

df_fetch = formfetch(t_list,ts_list)
print(df_fetch)

Successfully fetch data from twitter.
                DateTime                                              Event
0    2019-03-19 14:02:20  Accident on TPE (towards SLE) before Tampines ...
1    2019-03-19 13:52:20  Obstacle on AYE (towards Tuas) after Tuas Rd E...
2    2019-03-19 12:12:14  Accident on Dunearn Road (towards City) after ...
3    2019-03-19 11:01:42  Accident on Lower Delta Road towards Jalan Buk...
4    2019-03-19 10:39:32  Accident on PIE (towards Tuas) after Toa Payoh...
5    2019-03-19 10:27:51  Accident on Stevens Road towards Balmoral Park...
6    2019-03-19 09:54:32  Accident on Hill Street (towards Coleman Stree...
7    2019-03-19 09:46:48  Accident on KPE (towards ECP) after Tampines R...
8    2019-03-19 09:35:41  Accident on KPE (towards ECP) after Tampines R...
9    2019-03-19 09:10:04  Accident on PIE (towards Tuas) after Thomson R...
10   2019-03-19 09:00:05  Accident on PIE (towards Tuas) after Thomson R...
11   2019-03-19 08:44:04  Accident on PIE (towards

We will need to update the newly fetch data into the total data.

In [7]:
df_total = pd.read_csv("total_data.csv")
def updateData(_totaldf,_fetchdf):
    _cut_off = (_totaldf.loc[0]['DateTime'],_totaldf.loc[0]['Event'])
    _id_to_cut = _fetchdf[(_fetchdf['DateTime']==_cut_off[0])&(_fetchdf['Event']==_cut_off[1])].index.astype(int)[0]
    _pdf = _fetchdf.loc[0:_id_to_cut-2]
    _newdf = _totaldf.append(_pdf, sort=True)
    _newdf = _newdf.sort_values("DateTime",ascending=False)
    _newdf = _newdf.reset_index(drop=True)
    if len(_pdf)+len(_totaldf) == len(_newdf) :
        _newdf.to_csv("total_data.csv",encoding='utf-8',index=False)
        print("Total Database length is {}.".format(len(_newdf)))
    else:
        print("There's something wrong in updating dataframe, please check!")
    return _newdf

df = updateData(df_total,df_fetch)


Total Database length is 1202.


if we have already the data set, we will need to do some auto tagging. We will need to perform sentence cleaning and also extract respective address keyword in the events. 

In [8]:
import re

stopwords = ["near","/","along","between","end","on","towards","before","after","avoid","with","congestion","till","exit","exit.","closed","entrance","entrance.","at","right.","right","tunnel",'.']

def containWord(s,w):
    if (' '+w+' ') in (' '+s+' '):
        res = True
    elif (w+' ') in (' '+s+' '):
        res = True
    elif (w+' ') in (' '+s+' '):
        res = True
    else:
        res = False
    return res

def clean_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    return re.sub(r'\s{2,}',' ', sentence)

def keywordExt(s):
    add_l = []
    l_w = s.split(" ")
    ii = 0
    while (l_w[ii] != 'end'):
        if l_w[ii] in stopwords:
            add_e = ""
            while l_w[ii+1] not in stopwords:
                if len(add_e) == 0:
                    add_e = add_e+l_w[ii+1]
                else:
                    add_e = add_e+" "+l_w[ii+1]
                ii += 1
            if len(add_e) != 0 :
                add_l.append(add_e)
        ii += 1
    return add_l

def generateAdd(df):
    events = df['Event'].tolist()
    sep = "#"
    events = [ s.split(sep,1)[0] for s in events]
    events = [ clean_sentence(e) for e in events]
    addlist = [keywordExt(string+" end") for string in events]
    cleanlist = []
    for ii in range(len(addlist)):
        jj = 0
        add = []
        for jj in range(len(addlist[ii])):
            if not(containWord(addlist[ii][jj],'lane') or containWord(addlist[ii][jj],'lanes')):
                if addlist[ii][jj] == 'piechangi':
                    add.append('pie changi')
                elif addlist[ii][jj] == 'piejurong':
                    add.append('pie jurong')
                elif containWord(addlist[ii][jj],'amk'):
                    add.append('ang mo kio '+addlist[ii][jj].split(" ",1)[1])
                elif containWord(addlist[ii][jj],'tg'):
                    add.append('tanjong '+addlist[ii][jj].split(" ",1)[1])    
                else:
                    add.append(addlist[ii][jj])
            jj += 1
        cleanlist.append(add)
    # if lane is need, add addlist as address instead of cleanlist
    df['address'] = cleanlist
    #df.to_csv("test.csv",index=False)
    return df

df = generateAdd(df)


In [9]:
from geopy.geocoders import Nominatim,BANFrance
from geopy.exc import GeocoderTimedOut
addlist = df['address'].tolist()
def createAddplusCoor(addresslist):
    _add_book = dict()
    geolocator = Nominatim(user_agent="jimmy-app",timeout=2000)
    for ii in range(len(addresslist)):
        for jj in range(len(addresslist[ii])):
            try:
                _add_book[addresslist[ii][jj]]
            except KeyError:
                _loc = geolocator.geocode(addresslist[ii][jj]+" singapore")
                if _loc != None:
                    _add_book[addresslist[ii][jj]] = (_loc.latitude,_loc.longitude)
    return _add_book

try:
    ab = pd.read_csv("AddressBook.csv",index_col=0)
    address_book = dict([(i,(a,b)) for i,a,b in zip(ab.index,ab.latitude,ab.longitude)])
    print("Successfully found AddressBook and it's loaded!")
except IOError:
    print("Error! Address Book not found, proceed to create address book. Please wait!")
    address_book = createAddplusCoor(addlist)
    ab = pd.DataFrame.from_dict(address_book, orient='index', columns=['latitude','longitude'])
    ab.to_csv("AddressBook.csv")

assert(isinstance(address_book,dict))
print("The length of address book is {}.".format(len(address_book)))

Successfully found AddressBook and it's loaded!
The length of address book is 281.


Next, I proceed to assign coordinates to each event based on their keyword address. New address not in the address book will be updated.

In [10]:
#print(df.head())
import numpy as np

def coordinateTagging(df,address_book):
    addlist = df['address'].tolist()
    geolocator = Nominatim(user_agent="jimmy-app",timeout=2000)
    cord_l = []
    n_error = 0
    error_list = []
    for ii in range(len(addlist)):
        coord_e = []
        for jj in range(len(addlist[ii])):
            try:
                coord_e.append(address_book[addlist[ii][jj]])
            except KeyError:
                _loc = geolocator.geocode(addlist[ii][jj]+" singapore")
                if _loc != None:
                    address_book[addlist[ii][jj]] = (_loc.latitude,_loc.longitude)
                    coord_e.append(address_book[addlist[ii][jj]])
                else:
                    n_error += 1
                    if not(addlist[ii][jj] in error_list):
                        print("There's no coordinate found for {}.".format(addlist[ii][jj]))
                        error_list.append(addlist[ii][jj])
                    coord_e.append((np.nan,np.nan))
                    
        cord_l.append(coord_e)
    ab = pd.DataFrame.from_dict(address_book, orient='index', columns=['latitude','longitude'])
    ab.to_csv("AddressBook.csv")
    df['coordinate'] = cord_l
    return df,address_book,n_error

Fdf,new_address_book,error = coordinateTagging(df,address_book)

Fdf.to_csv("tagged.csv",encoding='utf-8',index=False)
print("There's {} number of error address in the data.".format(error))

There's no coordinate found for ctesle.
There's no coordinate found for ang mo kio avenue 5ang mo kio avenue 6 junction.
There's no coordinate found for sims way junction.
There's no coordinate found for chin swee roadclemenceau avenue junction.
There's no coordinate found for raffles boulevardtemasek avenue junction.
There's no coordinate found for clemenceau avenueriver valley road junction.
There's no coordinate found for pioneer rd nth.
There's no coordinate found for braddell rdlor chuan.
There's no coordinate found for pietuas.
There's no coordinate found for dunearn underpass.
There's no coordinate found for farrer flyover.
There's no coordinate found for 19 21 mar 2019 from midnight to 5 am.
There's no coordinate found for friday 15 march 2019.
There's no coordinate found for 10 pm.
There's no coordinate found for sunday 24 march 2019.
There's no coordinate found for wednesday 20 march 2019 and thursday 21 march 2019 from 130 am to 5 am daily.
There's no coordinate found for yu